In [11]:
# %%time
import pandas as pd
import requests
import numpy as np

# Get NSE Scrip expiry dates
expurl = "https://www.nseindia.com/live_market/dynaContent/live_watch/fomwatchsymbol.jsp?key=NIFTY&Fut_Opt=Futures"

exphtml = requests.get(expurl).content

fnotable = pd.read_html(exphtml, match='Expiry Date', header=0)[0]
fnoexpiry = fnotable['Expiry Date']

# Get the list of equity scrips
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
paisa = pd.read_html(paisaurl, header=0)[1]          # It's the second table in the url

# Remove VIX and NIFTY 
symbol = paisa.loc[~(paisa.Symbol.str.contains('VIX') | paisa.Symbol.str.contains('NIFTY'))].Symbol
symbol

# Combine Expiry and Scrips to a new url DataFrame
urldf = pd.concat([symbol] * len(fnoexpiry), keys=fnoexpiry).reset_index().drop('level_1', axis=1)

# NSE URLs
nseurl = "https://www.nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp?&instrument=OPTSTK&symbol="

# Make a Dataframe of the NSE URLs
urldf['URL'] = nseurl + urldf['Symbol'] + '&date=' + urldf['Expiry Date']
pd.set_option('display.max_colwidth', -1)

urldf = urldf.head() # Keep only the first 5 rows for brevity

chainheader = ['Call_OI', 'Call_OI_Chng', 'Call_Volume', 'Call_IV', 'Call_LTP', 'Call_NetChng', 'Call_BidQty', 'Call_BidPrice', 'Call_AskPrice', 'Call_AskQty', \
               'Strike Price', \
               'Put_BidQty', 'Put_BidPrice', 'Put_AskPrice', 'Put_AskQty', 'Put_NetChng', 'Put_LTP', 'Put_IV', 'Put_Volume', 'Put_OI_Chng', 'Put_OI']

def getchain(a, e, s):
    chainhtml = requests.get(a).content
    chain = pd.read_html(chainhtml)[1][:-1]  # read the first table and drop the total
    chain.columns=chain.columns.droplevel(0) # drop the first row of the header
    chain = chain.drop('Chart', 1)           # drop the charts
    
    chain['Expiry'] = e                      # expiry date
    chain['Symbol'] = s                      # symbol
    return chain

v = np.vectorize(getchain)
pd.concat(v(urldf.URL, urldf['Expiry Date'], urldf.Symbol))


# chain = getchain(a)
# chain.columns = chainheader
# urldf.loc[:, 'Chain']



,OI,Chng in OI,Volume,IV,LTP,Net Chng,BidQty,BidPrice,AskPrice,AskQty,...,AskPrice,AskQty,Net Chng,LTP,IV,Volume,Chng in OI,OI,Expiry,Symbol
0,56000,-,-,-,-,-,-,-,47.5,28000,...,3,56000,-,-,-,-,-,84000,25JAN2018,RCOM
1,-,-,-,-,-,-,28000,10.40,45.1,28000,...,3,56000,-,-,-,-,-,-,25JAN2018,RCOM
2,28000,-,-,-,-,-,-,-,42.4,28000,...,3,56000,-,-,-,-,-,1680000,25JAN2018,RCOM
3,56000,-,-,-,-,-,28000,5.40,40.1,28000,...,0.05,56000,-,-,-,-,-,1960000,25JAN2018,RCOM
4,392000,-,-,-,-,-,28000,3.00,37.4,28000,...,0.05,1.148e+06,-,-,-,-,-,5628000,25JAN2018,RCOM
5,1176000,-,-,-,-,-,28000,17.50,19.4,28000,...,0.05,2.604e+06,-,-,-,-,-,13888000,25JAN2018,RCOM
6,1372000,-,-,-,-,-,28000,1.15,32.15,28000,...,0.1,28000,-,0.10,173.38,1,-28000,3640000,25JAN2018,RCOM
7,1092000,-,-,-,-,-,28000,12.50,14.4,28000,...,0.15,532000,-0.05,0.10,142.02,2,-28000,6580000,25JAN2018,RCOM
8,784000,-,-,-,-,-,28000,9.35,25.35,28000,...,0.2,280000,-,-,-,-,-,2800000,25JAN2018,RCOM
9,1232000,-,-,-,-,-,28000,9.05,9.4,28000,...,0.45,56000,-0.05,0.40,123.35,3,28000,6720000,25JAN2018,RCOM


In [4]:
urldf

,Expiry Date,Symbol,URL
0,25JAN2018,RCOM,https://www.nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp?&instrument=OPTSTK&symbol=RCOM&date=25JAN2018
1,25JAN2018,RNAVAL,https://www.nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp?&instrument=OPTSTK&symbol=RNAVAL&date=25JAN2018
2,25JAN2018,JPASSOCIAT,https://www.nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp?&instrument=OPTSTK&symbol=JPASSOCIAT&date=25JAN2018
3,25JAN2018,RPOWER,https://www.nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp?&instrument=OPTSTK&symbol=RPOWER&date=25JAN2018
4,25JAN2018,IFCI,https://www.nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp?&instrument=OPTSTK&symbol=IFCI&date=25JAN2018
